# Social Computing/Social Gaming - Summer 2023
# Exercise Sheet 1 - Introduction to Python and Network Visualization
Welcome to the 2023 Social Computing / Social Gaming tutorial assignments. For all exercise sheets Python is the programming language of choice. This exercise sheet will therefore provide an introduction to Python for you. In the latter part of this exercise sheet you will take on your first Social Computing task.

In addition to the iPython notebooks, it is recommended to have a look at the introduction videos and/or the introduction slides, both provided on Moodle, for every exercise sheet as they contain **helpful hints** and the **form of the assignment** which is **mandatory**!

## Task 1.1: Sparse Vector Representation
To get you prepared for using Python in the upcoming exercise sheets we will introduce some of Python's most important features on the example of sparse vectors. The basic idea is that sparse vectors (vectors whose positions are mostly zero) can be represented more efficiently by only saving a list of tuples containing the valueable data's position and its data. For example (0,0,0,4,0,0,42) would be represented by [(3,4),(6,42)].

In [ ]:
#Your example data
ExampleVector1 = #TODO
ExampleVector2 = #TODO


**a)** As the first step, **complete** the ``sparseVectorTransform()`` function so that it transforms the given vector to a list of tuples as stated in the example above. Create reasonable ``ExampleVectors`` in the block above to test your function.

**Hint:** You can use ``enumerate()`` in a for-loop. You can also use print(type(variable)) if u want to know the type.

In [ ]:
def sparseVectorTransform(vector):

    #TODO:

**b)** In Python variables are not assigned to a fixed datatype in contrast to Java for example. Resulting from this, lists for example can contain different datatypes for its element even a function can be stored in a variable or list!
Below you can see ``combineSVs()`` which combines two of our sparse vectors in tupel representation into one by only considering positions that are set in both vectors. The data entries for the respective position are combined with use of the handed over function. As you can see the function is handed over just like regular parameters in Python.

In [ ]:
def combineSVs(func:callable,SV1:list,SV2:list)-> list:
    returnlist = []
    for element1 in SV1:
        for element2 in SV2:
            if element1[0] == element2[0]: returnlist.append( (element1[0], func(element1[1],element2[1])))
    return returnlist

It is now your task to **define** such a combining function to hand over to the `combineSVs()`. As we will only know at runtime what datatypes are contained in the vector, this function first needs to check which datatypes the given parameters have. For this task we will allow the vectors to contain Integers, Strings and Functions.
Depending on datatypes, the result for their combination should be:
- Integer & Integer => Add both Integers
- String & String => Concatenate both String sequences
- String & Integer => Add the Integer as character at the end of the String
- Integer & String => Multiply the String sequence by Integer.
- Function & (String||Integer) => The result of the Function with the String or Integer as its parameter

**Hint:** https://docs.python.org/3/library/functions.html#isinstance
https://docs.python.org/3/library/functions.html#callable


In [ ]:
def combineElements (element1,element2):
    types = ['type1','type2']
    
    
    #TODO: Assign the datatypes of the elements to types

    
    
    #TODO: Create the correct result regarding the datatypes in types and return it

    
    
    #If the elements' datatypes do not fit any of our combination this exception will be thrown
    raise TypeError("Zulässige Kombinationen der Datentypen nicht eingehalten!")

**c)** The vectors below contain all possible combinations specified above. Use them to **compute** your final combined sparse vector in tuple representation. Then shortly **explain** in your own words the chances and pitfalls of Python's datatype independent variables. Three sentences will be sufficient.

In [ ]:
v1 = (1,2,'Hallo Welt',len)
v2 = (3,'Hello world',4,'123')
print(combineSVs(combineElements, sparseVectorTransform(v1), sparseVectorTransform((v2))))

As example (0,0,0,4,0,0,42) and (0,0,0,4,0,0,42) with print(combineSVs(combineElements, sparseVectorTransform((0,0,0,4,0,0,42)), sparseVectorTransform(((0,0,0,4,0,0,42))))) results in: [(3, 8), (6, 84)]



**TODO: Write your explanation here:**

## Task 1.2: Twitch Social Network with NetworkX

For your second task you will now take your first steps into Social Computing / Social Gaming. We will therefore have a look at German streamers from [Twitch](https://www.twitch.tv/) [1]. Twitch is an international platform for live-streaming games or other content. The data set [2] has been created in 2019 and contains lots of information about different streamers. 
For our tasks we will only consider the German Twitch-streamers. Twitch offers lots of functionalities including a friendship system. The task is to draw a graph representing the **friendship network** of the hundred most watched German streamers.

In [ ]:
#Import the required libraries
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

**Read** in the data set by using the **Pandas** library [3]. For a quick overview, you can refer to [4]. Pandas provides a powerful data structure called the ''DataFrame'' which is widely used in data science and machine learning. For your exercise sheets it will always come in handy to have a look at the used package's **documentations** online!

**Inspect** the three dataframes to fully understand what they contain. Don't confuse the Twitch IDs (``id``) with the node's IDs for our data set (``new_id``), which will form the key for our dataset.

**Hint:** You can have a look at the first entries of your Pandas data-frames with `Dataframe.head(10)`.

In [ ]:
#Contains a multitude of information about the streamers (adressed by their Twitch IDs).
nodes_DF = pd.read_csv('musae_DE_target.csv')
#Contains the friendship relations between streamers.
edges_DF = pd.read_csv('musae_DE_edges.csv')
#Contains the names of the top 100 Twitch-streamer for their Twitch IDs.
twitchNames_DF = pd.read_csv('TwitchIDList.csv')

**a)** First, we will deal with our graph's nodes, the streamers. 
We are only interested in streamers who have been streaming for more than 2000 days. **Filter** the data to contain only such rows.
There are some columns we will not need later. **Remove** the unnecessary information by **dropping** the `id`, ``days`` and ``mature`` columns from ``nodes_DF``. 
Then, to limit our data on the 100 most viewed streamers **order** ``nodes_DF`` by ``views`` and only **keep** the first 100 entries as ``orderedNodes_DF``.

In [ ]:
#TODO:

**b)** Now we need to work on the friendship data-frame, our graph's edges. As we will only draw the top 100 streamers as nodes, we are only interested in friendship relations in between these 100 nodes. The relations are represented as "from"->"to" even though the friendship relation is considered undirected.

**Fill** the list  ``drop_indices`` with all entry's indices that have their target outside of our 100 nodes. Therefore go through all entries of `edges_DF` and check if their ``to`` value is contained in ``nodesList``.
Then **drop** all the respective entries from our data-frame.

**Hint:** We do not need to worry about the ``from`` values as our merging process will take care of that part in the next task.

In [ ]:
drop_indices = []
nodesList = orderedNodes_DF['new_id'].tolist()

#TODO:


**c)** For the next step, we now have to **merge** our dataframes. This works similar to a SQL Join operation. We want to use an **'inner' merge** (similar to an "inner"=="normal" join in SQL) and make sure the nodes ID matches the friendship relation's origin. Thus it will only consider any friendships of our 100 nodes.


SQL-join:
https://www.w3schools.com/sql/sql_join.asp

In [ ]:
merged_DF = #TODO

Now networkX [5] **reads** all the edges from our dataframe and creates a Graph object. **Take a look** at the number of nodes in your graph. **Explain** your observations shortly.

In [ ]:
graph = nx.from_pandas_edgelist(merged_DF, "new_id", "to")

#TODO: Take a look at the number of nodes


**TODO: Write your explanation here:**

**d)** Before we finally draw our graph, lets add some **additional information** to it. We want to represent if the respective streamer is an official Twitch-partner by the **color** of a node.  The ``partner`` column in our dataframe will provide you with the partner-status. 

All entries were represented by their ID until now. To make our graph readable we want to name all nodes with the streamers' actual names that are contained in ``twitchNames_DF``. You have now to **create two dictionaries** that map the nodes ID to its partner state and streamer's name.

Thereafter the given code will create the colormap from your dictionary, that will later be used to apply the colors to our graph. Then the graph object is created.

**Hint:** As ``twitchNames_DF`` only contains the most 100 viewed streamer's names you can use it as an indicator whether you have solved everything correctly.

In [ ]:
partnerState_dict = #TODO
twitchNames_dict = #TODO

colormap = []
for node in graph:
    if(partnerState_dict[node] == True):
        colormap.append('Purple')
    else:
        colormap.append('Blue')

graph = nx.relabel_nodes(graph, twitchNames_dict)

**e)** Now to conclude your first exercise sheet you can finally draw the graph. NetworkX offers you multiple pre-defined **[graph layouts](https://networkx.org/documentation/stable/reference/drawing.html#module-networkx.drawing.layout)** [6] that you can use. Don't forget to make use of the ``colormap`` and the set options! Matplotlib will then print your graph.

**Try** out the different layouts and chose two that fit the graph well. Then **explain** why they fit well and what **observations** you could draw from our visualized network shortly!

In [ ]:
plt.figure(1,figsize=(40,40)) 
options = {
    "font_size" : 30,
    "edge_color": 'lightGrey'
    }

#TODO: Draw the network. Hint: Use the draw_networkx method instead of draw.


plt.show()

**TODO: Write your explanation and observations here:**

## References

<br>[1] https://www.twitch.tv/
<br>[2] Rozemberczki, B., Allen, C. & Sarkar, R. (2019). *Multi-scale Attributed Node Embedding*
<br>[3] https://pandas.pydata.org/docs/getting_started/index.html
<br>[4] https://pandas.pydata.org/docs/user_guide/10min.html
<br>[5] https://networkx.org/documentation/stable/reference/index.html
<br>[6] https://networkx.org/documentation/stable/reference/drawing.html#module-networkx.drawing.layout